### Exp: 1 working with Non-snapshot

Assumption: every same vehicle in the same parking timespan have a customer that belongs to that car

Vehicle : check , same type, same cam , timestamp in the lifespan +- grace windows (padding periods) x,y is near each other ~~appear at least 2 timeframe~~ on the same x,y on non-snapshot cam

OUTPUT : vehicle_index : {timestamp range, x, y, vehicle type, }

camera used : mbk-14-12-vehicle 

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger.Approch1 import match_cross_to_vehicle, match_reverse_to_vehicle

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

CURRENT_CAM = "mbk-14-11"
SAVE_DIR = os.path.join(conf.RESOURCES_PROCESSED_DIR, f'2024-04-29/count-passengers-{CURRENT_CAM}-vehicle.csv')

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-29/mbk-tourist-vehicle-object-20240429.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-29/mbk-tourist-raw-cross-object-20240429.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-29/mbk-tourist-raw-reverse-object-20240429.csv")
)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT]].to_string())
df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)

# df_vehicle[conf.VEHICLE_INTEREST_NON_SNAPSHOT]

# drop the one that lifetime is less than 4 seconds
# df_vehicle = df_vehicle.loc[(df_vehicle["lifetime"] > 4),]

# create a timestamp_unix at the end of lifespan
# df_vehicle["timestamp_unix_end"] = df_vehicle["timestamp_unix"] + (df_vehicle["lifetime"].astype(np.int64))
# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT,'timestamp_unix','timestamp_unix_end']].to_string())

## Cross vs Vehicle

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

         camera       timestamp_precise    xmid    ymid  timestamp_unix
2289  mbk-14-11 2024-04-29 02:03:45.182  1831.5   394.0      1714356225
2291  mbk-14-11 2024-04-29 02:03:47.662  1290.5   704.5      1714356227
2290  mbk-14-11 2024-04-29 02:03:49.301  1806.0   403.5      1714356229
2262  mbk-14-11 2024-04-29 02:29:07.936  2026.0   278.0      1714357747
2263  mbk-14-11 2024-04-29 02:29:24.451  2079.0   285.5      1714357764
2264  mbk-14-11 2024-04-29 02:29:25.052  2114.0   160.0      1714357765
2260  mbk-14-11 2024-04-29 02:29:31.769  1988.5   377.5      1714357771
2265  mbk-14-11 2024-04-29 02:29:47.925  2058.0   287.5      1714357787
2238  mbk-14-11 2024-04-29 02:45:54.510  2168.0   220.0      1714358754
2239  mbk-14-11 2024-04-29 02:48:51.941  2098.5   223.5      1714358931
2235  mbk-14-11 2024-04-29 02:50:26.998  2064.0   251.0      1714359026
2217  mbk-14-11 2024-04-29 02:54:58.682  2122.5   135.0      1714359298
2218  mbk-14-11 2024-04-29 02:55:02.885  2113.0   135.5      171

## Reverse vs Vehicle

In [3]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

## TOGETHER


In [4]:
df_vehicle["lifetime"]

1563     2.599
1551     3.479
1545     1.399
1544    31.711
1540    51.625
         ...  
193      1.718
194      8.238
106      1.760
24      30.871
8        2.000
Name: lifetime, Length: 190, dtype: float64

In [5]:
ken = match_reverse_to_vehicle(df_reverse, df_vehicle)
johnny = match_cross_to_vehicle(
    df_cross,
    ken,
)


# df_vehicle = df_vehicle[
#     (df_vehicle["timestamp_unix"] >= 1714284000) & (df_vehicle["timestamp_unix_end"] <= 1714288020)
# ]
# df_vehicle = df_vehicle[
#     [
#         *conf.VEHICLE_INTEREST_NON_SNAPSHOT,
#         "cross_count",
#         "reverse_count",
#         "original_video_name",
#         # "frame_no",
#     ]
# ]

# df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)
# print(
#     df_vehicle[
#         [
#             "camera",
#             "timestamp_precise",
#             "lifetime",
#             "cross_count",
#             "reverse_count",
#             "vehicle_type",
#             "plate_number",
#             "timestamp_unix",
#             "timestamp_unix_end",
#             "xmid",
#             "ymid",
#         ]
#     ].to_string()
# )

In [6]:
johnny.to_csv(path_or_buf=SAVE_DIR, index = False)

## TODO 
- check cross walk angle?
- allow customer able to be classified to the next car (not found in the current time stamp)?
- check displacement?